In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

## Load BERT embeddings for the classifier input

In [ ]:
# final output(45000)

train_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\Embeddings\\45000_epoch_5\\final_train_outputs_reshaped.pt")
test_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\Embeddings\\45000_epoch_5\\final_eval_outputs_reshaped.pt")

train_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\Embeddings\\45000_epoch_5\\final_train_output_labels.pt")
test_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\Embeddings\\45000_epoch_5\\final_eval_output_labels.pt")

In [ ]:
# final output 512 (45000)

train_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_1_train_outputs_reshaped.pt")
test_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_1_eval_outputs_reshaped.pt")

train_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_train_output_labels.pt")
test_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_eval_output_labels.pt")

In [ ]:
# final output 512 (45000)

train_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_train_outputs_logits_reshaped.pt")
test_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_eval_outputs_logits_reshaped.pt")

train_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_train_output_labels.pt")
test_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\512_eval_output_labels.pt")

In [ ]:
# output from training on 42601 items with more options: we have unsqueezed labels and large validation set

train_features = torch.load(r"C:\Users\ivank\Documents\BERT_projects\Embeddings\42601_new\new_train_outputs_reshaped.pt")
test_features = torch.load(r"C:\Users\ivank\Documents\BERT_projects\Embeddings\42601_new\new_eval_outputs_reshaped.pt")

train_labels = torch.load(r"C:\Users\ivank\Documents\BERT_projects\Embeddings\42601_new\new_train_output_labels.pt")
test_labels = torch.load(r"C:\Users\ivank\Documents\BERT_projects\Embeddings\42601_new\new_eval_output_labels.pt")

In [ ]:
val_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\new_validation_outputs_reshaped.pt")
#val_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\new_validation_outputs_logits_reshaped.pt")
val_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\new_validation_output_labels.pt")

In [ ]:
large_val_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\new_large_validation_outputs_reshaped.pt")
#large_val_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\new_large_validation_outputs_logits_reshaped.pt")
large_val_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\new_large_validation_output_labels.pt")

## For using per-sequence input, flatten the features and load flattened labels

In [ ]:
test_labels[:10]

In [ ]:
test_labels

In [ ]:
train_labels = torch.load("C:\\Users\ivank\\Documents\\BERT_projects\\Embeddings\\Unsqweezed labels\\unsqueezed_train_output_labels.pt")
test_labels = torch.load("C:\\Users\ivank\\Documents\\BERT_projects\\Embeddings\\Unsqweezed labels\\unsqueezed_eval_output_labels.pt")
#val_labels = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\unsqueezed_validation_output_labels.pt")

In [ ]:
train_labels = train_labels.to(device)
test_labels = test_labels.to(device)

In [ ]:
train_labels = torch.FloatTensor(train_labels).to(device)
test_labels = torch.FloatTensor(test_labels).to(device)

In [ ]:
# compare to verify using the same data

str(test_labels[:300])

In [ ]:
train_features = torch.reshape(train_features, (1278180, 768))
test_features = torch.reshape(test_features, (142020, 768))
#val_features = torch.reshape(val_features, (157830, 768))

In [ ]:
train_features = torch.reshape(train_features, (1350000, 768))
test_features = torch.reshape(test_features, (228030, 768))

## Load untuned embeddings

In [ ]:
train_features = torch.load("C:\\Users\\ivank\\Documents\\BERT_projects\\Embeddings\\Untuned\\final_train_outputs_reshaped.pt")
#train_labels = torch.load("C:\\Users\ivank\\Documents\\BERT_projects\\Embeddings\\Unsqweezed labels\\unsqueezed_train_output_labels.pt")

#for untuned

In [ ]:
train_features = torch.reshape(train_features, (1350000, 768))
#train_labels = train_labels.to(device)

#for untuned

## For training a model without pooling, apply averaging to features. Labels don't need to be changed

In [ ]:
train_features = torch.mean(train_features, axis = 1)
test_features = torch.mean(test_features, axis = 1)
#val_features = torch.mean(val_features, axis = 1)

## Analyze input embeddings: see the difference by label

In [ ]:
train_all = torch.cat((train_features, train_labels.reshape(-1,1)), axis=1)

In [ ]:
train_all = train_all.cpu().numpy()

In [ ]:
"""To analyze test features weights"""

#train_all = torch.cat((test_features, test_labels.reshape(-1,1)), axis=1)
#train_all = train_all.cpu().numpy()

In [ ]:
import pandas as pd

train_dataframe = pd.DataFrame(train_all)

In [ ]:
train_dataframe_neg = train_dataframe[train_dataframe[768] == 0.0]
train_dataframe_pos = train_dataframe[train_dataframe[768] == 1.0]
train_dataframe_neg = train_dataframe_neg.drop(columns=[768])
train_dataframe_pos = train_dataframe_pos.drop(columns=[768])


In [ ]:
del train_dataframe

In [ ]:
pos_mean = train_dataframe_pos.mean(axis = 0)
neg_mean = train_dataframe_neg.mean(axis = 0)

In [ ]:
pos_std = train_dataframe_pos.std(axis = 0)
neg_std = train_dataframe_neg.std(axis = 0)

In [ ]:
diff = pos_mean - neg_mean
absdiff = abs(diff)
diff_sort = absdiff.sort_values(ascending=False)
pos_sort = pos_mean.sort_values()
neg_sort = neg_mean.sort_values()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(pos_std)), pos_std, width=1, edgecolor=(0, 0, 0), color = 'lime', label = "Feature weights STD for class 1")
plt.bar(range(len(pos_std)), neg_std, width=1, edgecolor=(0, 0, 0), color = 'bisque', label = "Feature weights STD for class 0")

plt.xlabel("Features")
plt.ylabel("Standard deviation")
plt.title("Standard deviation of feature weights in all segments with labels '1' and '0'")
plt.legend(fontsize=12)

plt.show()

In [ ]:
fig7, ax7 = plt.subplots()
ax7.set_title('Averaged feature weights for untuned train, tuned train, and tuned test data (each for classes "1" and "0")')
ax7.boxplot([pos_mean, neg_mean, pos_mean_train, neg_mean_train, pos_mean_test, neg_mean_test])

plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(pos_mean)), pos_mean, width=1, edgecolor=(0, 0, 0), label = "Feature weights for class 1")
plt.bar(range(len(neg_mean)), neg_mean, width=1, edgecolor=(0, 0, 0), color = 'peachpuff', label = "Feature weights for class 0")

plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Average weights of features in all segments with labels '1' and '0'")
plt.legend(fontsize=12)

plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(pos_mean)), pos_mean, width=1, edgecolor=(0, 0, 0), label = "Feature weights for class 1")

plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Average weights of features in all segments with labels '1' in untuned model")
plt.legend(fontsize=12)

plt.show()

plt.figure(figsize=(30, 10))
plt.bar(range(len(neg_mean)), neg_mean, width=1, edgecolor=(0, 0, 0), color = 'peachpuff', label = "Feature weights for class 0")

plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Average weights of features in all segments with labels '0' in untuned model")
plt.legend(fontsize=12)

plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(pos_sort)), pos_sort, width=1, edgecolor=(0, 0, 0))
#plt.bar(range(len(neg_sort)), neg_sort, width=1, edgecolor=(0, 0, 0), color = 'peachpuff', label = "Total top-feature counts for 0")

plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Average weights of features in all segments with label '1', sorted")
plt.legend(fontsize=12)
plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(diff_sort)), diff_sort, width=1, edgecolor=(0, 0, 0))

plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Absolute difference in feature weights between classes, sorted")
plt.legend(fontsize=12)
plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(diff_sort)), diff_sort, width=1, edgecolor=(0, 0, 0))
y = np.array([2, 4, 6, 8, 9, 10, 12, 14, 16])


plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Absolute difference in feature weights between classes, sorted")
plt.legend(fontsize=12)
plt.yticks(np.arange(0, 5, 1.0))
plt.show()

Checking the number of overlapping features between most popular extracted from LIME and most salient in value difference

In [ ]:
# Using the saliency values (absolute difference in weights between classes) from 45000 training dataset [without averaging sequences into texts]
# and LIME explanations from PyTorch classifier on 7601 validation data ("test data") and 45000 training data ("training data"):

# training data in LIME / training data in saliency after finetuning:
# {130, 382, 517, 135, 137, 638, 653, 275, 153, 412, 416, 294, 553, 297, 47, 566, 438, 585, 715, 215, 216, 222, 478, 498, 243, 766} 
# 52.0% of features are overlapping

# test data in LIME / training data in saliency after finetuning:
# {382, 389, 137, 522, 524, 653, 275, 276, 277, 416, 553, 297, 312, 443, 455, 585, 715, 205, 598, 216, 95, 745, 498, 375, 766} 25 
# 50.0% of features are overlapping (numbers were up to 57% for 40 most popular for each class)

# trainin data in LIME / training data in saliency with untuned embeddings
# {176, 758, 153, 222, 286} 
# 10.0% of features are overlapping

# test data in LIME / training data in saliency with untuned embeddings
# {312, 620, 277} 
# 6.0% of features are overlapping

# training data in LIME / test data in saliency after finetuning:
# {130, 382, 517, 135, 137, 638, 653, 275, 153, 412, 416, 294, 553, 297, 47, 563, 566, 438, 585, 715, 215, 216, 222, 478, 498, 243, 766} 
# 54.0% of features are overlapping

# test data in LIME / test data in saliency after finetuning:
# {382, 389, 137, 522, 524, 653, 275, 276, 277, 416, 553, 297, 312, 443, 455, 585, 715, 205, 216, 95, 745, 498, 375, 766} 
# 48.0% of features are overlapping

n = 50

diff_dict = diff_sort[:n].to_dict()

#old, test data

"""top_pos_lime_dict = {553: 51, 522: 35, 95: 33, 455: 31, 297: 27, 715: 21, 416: 18, 216: 17, 766: 14, 389: 13, 556: 13, 277: 12, 276: 10, 152: 9, 598: 9, 382: 9, 498: 8, 
137: 8, 585: 7, 289: 7, 36: 6, 524: 6, 195: 6, 85: 5, 461: 5, 746: 5, 100: 5, 559: 5, 502: 4, 384: 4, 443: 4, 205: 4, 125: 3, 138: 3, 31: 3, 438: 3, 673: 3, 653: 3, 582: 2, 607: 2,
 724: 2, 745: 2, 127: 2, 44: 2, 754: 2, 375: 2, 554: 2, 345: 2, 532: 2, 533: 2, 381: 2, 635: 2, 579: 2, 312: 2, 292: 2, 218: 2, 437: 2, 275: 2, 751: 2, 96: 2, 566: 2, 729: 2, 620: 2, 
 227: 2, 322: 1, 563: 1, 494: 1, 158: 1, 426: 1, 298: 1, 290: 1, 704: 1, 758: 1, 288: 1, 574: 1, 485: 1, 517: 1, 244: 1, 66: 1, 87: 1, 29: 1, 13: 1, 2: 1, 698: 1, 611: 1, 108: 1,
  113: 1, 243: 1, 173: 1, 399: 1, 454: 1, 613: 1, 240: 1, 733: 1, 341: 1, 86: 1, 222: 1, 626: 1, 471: 1, 519: 1, 77: 1, 428: 1, 667: 1, 463: 1, 712: 1, 258: 1, 175: 1, 293: 1, 752: 1, 
59: 1, 54: 1, 615: 1, 565: 1, 738: 1, 135: 1, 475: 1, 376: 1, 342: 1, 534: 1, 725: 1, 181: 1, 98: 1, 50: 1, 130: 1, 164: 1, 153: 1, 5: 1, 308: 1, 360: 1, 144: 1, 422: 1, 284: 1, 198: 1}

top_neg_lime_dict = {553: 31, 455: 29, 95: 26, 522: 23, 389: 22, 297: 20, 715: 20, 416: 14, 216: 12, 382: 11, 277: 10, 556: 9, 559: 8, 766: 7, 137: 7, 312: 6, 152: 6, 
276: 6, 85: 5, 36: 5, 275: 4, 375: 4, 289: 4, 402: 4, 215: 3, 673: 3, 751: 3, 471: 3, 329: 3, 585: 3, 195: 3, 620: 3, 524: 3, 443: 3, 598: 3, 381: 2, 127: 2, 653: 2, 540: 2, 502: 2,
 66: 2, 175: 2, 218: 2, 53: 2, 164: 2, 474: 2, 41: 2, 399: 2, 712: 2, 498: 2, 100: 2, 441: 2, 745: 2, 59: 2, 138: 2, 516: 2, 205: 2, 461: 2, 563: 1, 13: 1, 463: 1, 475: 1, 239: 1, 
 344: 1, 353: 1, 188: 1, 144: 1, 31: 1, 110: 1, 557: 1, 737: 1, 532: 1, 0: 1, 408: 1, 32: 1, 725: 1, 512: 1, 286: 1, 658: 1, 161: 1, 393: 1, 676: 1, 404: 1, 572: 1, 348: 1, 724: 1, 
 582: 1, 384: 1, 321: 1, 499: 1, 194: 1, 309: 1, 20: 1, 372: 1, 328: 1, 367: 1, 153: 1, 255: 1, 636: 1, 200: 1, 73: 1, 185: 1, 523: 1, 517: 1, 298: 1, 351: 1, 258: 1, 413: 1, 135: 1, 
 593: 1, 478: 1, 345: 1, 212: 1, 130: 1, 60: 1, 149: 1, 278: 1, 44: 1, 354: 1, 292: 1, 533: 1, 223: 1, 141: 1, 597: 1, 534: 1, 424: 1, 547: 1, 727: 1, 469: 1, 181: 1, 226: 1, 703: 1, 357: 1, 635: 1, 158: 1, 392: 1}"""

#new, training data
top_pos_lime_dict = {135: 44, 498: 32, 47: 28, 553: 22, 216: 21, 297: 17, 585: 17, 153: 16, 137: 16, 766: 14, 294: 13, 275: 12, 638: 12, 727: 11, 715: 11, 758: 10, 416: 10, 517: 8, 502: 8, 378: 8, 
653: 7, 345: 7, 130: 6, 596: 6, 222: 6, 532: 6, 412: 6, 646: 6, 608: 6, 114: 6, 127: 5, 635: 5, 382: 5, 241: 4, 215: 4, 66: 4, 295: 4, 243: 4, 401: 4, 729: 4, 566: 3, 286: 3, 438: 3, 624: 3, 265: 3, 613: 3, 
253: 3, 708: 3, 85: 3, 478: 3, 492: 3, 522: 2, 573: 2, 176: 2, 540: 2, 121: 2, 304: 2, 421: 2, 159: 2, 337: 2, 484: 2, 178: 2, 563: 2, 361: 2, 181: 2, 334: 2, 661: 2, 523: 1, 289: 1, 1: 1, 639: 1, 
544: 1, 480: 1, 166: 1, 357: 1, 156: 1, 16: 1, 651: 1, 163: 1, 761: 1, 543: 1, 597: 1, 11: 1, 524: 1, 384: 1, 161: 1, 360: 1, 175: 1, 410: 1, 724: 1, 172: 1, 106: 1, 226: 1, 701: 1, 329: 1, 691: 1, 
168: 1, 647: 1, 154: 1, 35: 1, 736: 1, 533: 1, 618: 1, 255: 1, 399: 1, 270: 1, 636: 1, 370: 1, 276: 1, 108: 1, 31: 1, 94: 1, 753: 1, 408: 1, 299: 1, 198: 1, 245: 1}

top_neg_lime_dict = {135: 25, 553: 22, 498: 20, 294: 16, 297: 14, 47: 13, 608: 13, 137: 12, 766: 12, 275: 11, 758: 10, 416: 10, 585: 10, 153: 9, 216: 8, 638: 8, 540: 7, 215: 7, 378: 7, 
176: 7, 596: 6, 517: 6, 624: 6, 653: 6, 85: 6, 127: 6, 295: 6, 94: 6, 712: 5, 243: 5, 502: 5, 727: 5, 532: 5, 635: 4, 130: 4, 412: 4, 566: 4, 159: 4, 729: 4, 444: 4, 646: 4, 401: 3, 114: 3, 478: 3, 
426: 3, 438: 3, 241: 3, 563: 3, 286: 3, 222: 3, 636: 2, 715: 2, 745: 2, 337: 2, 484: 2, 265: 2, 181: 2, 724: 2, 108: 2, 528: 2, 625: 2, 676: 2, 691: 2, 31: 2, 345: 2, 615: 1, 4: 1, 370: 1, 708: 1, 
524: 1, 658: 1, 329: 1, 522: 1, 685: 1, 687: 1, 173: 1, 255: 1, 273: 1, 387: 1, 321: 1, 382: 1, 379: 1, 665: 1, 324: 1, 251: 1, 562: 1, 572: 1, 253: 1, 357: 1, 629: 1, 613: 1, 603: 1, 259: 1, 573: 1, 
556: 1, 597: 1, 258: 1, 66: 1, 10: 1, 761: 1, 58: 1, 225: 1, 178: 1, 375: 1, 460: 1, 583: 1, 399: 1, 334: 1, 516: 1, 72: 1, 592: 1, 747: 1}

total_lime_dict = {k: top_pos_lime_dict.get(k, 0) + top_neg_lime_dict.get(k, 0) for k in set(top_pos_lime_dict) | set(top_neg_lime_dict)}
total_lime_dict = dict(sorted(total_lime_dict.items(), key=lambda item: item[1], reverse=True))

common_features = set(diff_dict.keys()).intersection(set(list(total_lime_dict.keys())[:n]))
#print(common_features, "\n{}% of features are overlapping".format(len(common_features)/((len(diff_dict)+len(total_lime_dict))/2)*100))
print(common_features, "\n{}% of features are overlapping".format((len(common_features)/n)*100))

## Initializing the model

In [ ]:
"""
Initialize the dataloaders. Use True for training, False for getting predictions
"""

train_tensor = TensorDataset(train_features, train_labels) 
train_dataloader = DataLoader(train_tensor, shuffle=True, batch_size=30)

test_tensor = TensorDataset(test_features, test_labels) 
eval_dataloader = DataLoader(test_tensor, shuffle=True, batch_size=30)

In [ ]:
"""
Tool for comparing the ouput of Torch average pooling and mean functions
"""

i = 0       
for batch in train_dataloader:
    i += 1
    m = nn.AdaptiveAvgPool2d((1,768))
    #input = torch.rand(64, 30, 768)
    input = batch[0]
    print(input.size())
    output = m(input)
    output = torch.flatten(output, 1)
    print("tensor1", output.size())
    
    output2 = torch.mean(input, axis = 1)
    print("tensor2", output2.size())

    for j in range(768):
        if not (torch.eq(output[0][j-1], output2[0][j-1])):
            print(output[0][j-1] - output2[0][j-1])
    if i == 1:
        break


In [ ]:
class SimpleNet(nn.Module):
    """
    A simple model with structure identical to FinalNet
    """

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(768, 16)
        self.fc2 = nn.Linear(16,2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class FinalNet(nn.Module):
    """
    A Sequential model without pooling (use with averaged input)
    """

    def __init__(self, num_classes=2):
        super(FinalNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(768, 16),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

In [ ]:
class FinalNetAvg(nn.Module):
    """
    A Sequential model with pooling (use with raw input)
    """

    def __init__(self, num_classes=2):
        super(FinalNetAvg, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 768))
        self.classifier = nn.Sequential(
            nn.Linear(768, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
class FinalNetAvg(nn.Module):
    """
    A Sequential model with pooling (use with raw input)
    """

    def __init__(self, num_classes=2):
        super(FinalNetAvg, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(768, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
class TwoAvg(nn.Module):
    """
    A Sequential model with pooling (use with raw input)
    """

    def __init__(self, num_classes=2):
        super(TwoAvg, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 2))
        self.classifier = nn.Sequential(
            nn.Linear(2, num_classes)
        )

    def forward(self, x):
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
class JustAvg(nn.Module):
    """
    A Sequential model with pooling (use with raw input)
    """

    def __init__(self, num_classes=2):
        super(JustAvg, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 2))

    def forward(self, x):
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

In [ ]:
model = JustAvg()

In [ ]:
model = TwoAvg()

In [ ]:
model = FinalNetAvg()

In [ ]:
model = SimpleNet()

In [ ]:
test_features[0]

In [ ]:
model.eval()
with torch.no_grad():
    output = model(torch.unsqueeze(test_features[0], 0))

## Train the model

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.5f}, Validation Loss: {:.5f}, accuracy = {:.5f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
    
model.to(device)

In [ ]:
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_dataloader, eval_dataloader, epochs=10, device=device)

In [ ]:
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_dataloader, eval_dataloader, epochs=10, device=device)
#512

In [ ]:
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_dataloader, eval_dataloader, epochs=10, device=device)
#512-4

In [ ]:
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_dataloader, eval_dataloader, epochs=10, device=device)
#512-logits

In [ ]:
train(model, optimizer, torch.nn.CrossEntropyLoss(), train_dataloader, eval_dataloader, epochs=2, device=device)
#512-logits

## Evaluate the model

In [ ]:
from datasets import load_metric


def eval(model, loss_fn, val_loader, device="cpu"):

    metric1 = load_metric("accuracy")
    metric2 = load_metric("f1")

    num_correct = 0 
    num_examples = 0
    valid_loss = 0.0
    
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():

            inputs, targets = batch
            inputs = inputs.to(device)
            logits = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(logits,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            predictions = torch.argmax(logits, dim=-1)

            metric1.add_batch(predictions=predictions, references=targets)
            metric2.add_batch(predictions=predictions, references=targets)

            correct = torch.eq(torch.max(F.softmax(logits, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
    valid_loss /= len(val_loader.dataset)

    return(metric1.compute(), metric2.compute(), valid_loss, num_correct / num_examples)


In [ ]:
eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)

In [ ]:
eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)
#512

In [ ]:
eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)
#512 logits JustAvg Best

In [ ]:
eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)
#512 logits JustAvg

In [ ]:
eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)
#512 logits no training

In [ ]:
weight = model.state_dict()['classifier.0.weight']

#-0.1429, -0.36, -0.128, 0.6

In [ ]:
torch.save(model.state_dict(), r"C:\Users\ivank\Documents\BERT_projects\Classifier\model_simple.pth")

## Load the model from file

In [ ]:
model = FinalNetAvg()
model.load_state_dict(torch.load(r"C:\Users\ivank\Documents\BERT_projects\Classifier\model_45000.pth"))
model.to(device)
#eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)

In [ ]:
model = SimpleNet()
model.load_state_dict(torch.load(r"C:\Users\ivank\Documents\BERT_projects\Classifier\model_simple.pth"))
model.to(device)
eval(model, torch.nn.CrossEntropyLoss(), eval_dataloader, device=device)

In [ ]:
model


In [ ]:
weight = model.state_dict()['fc1.weight']

In [ ]:
weight = model.state_dict()['classifier.2.weight']

In [ ]:
print(weight)

In [ ]:
weight = weight.mean(axis = 0)

In [ ]:
weightdf = pd.DataFrame(weight.cpu().numpy())

In [ ]:
weight_and_posneg = weightdf.append(pos_mean, ignore_index=True)

In [ ]:
weight_and_posneg = weight_and_posneg.append(neg_mean, ignore_index=True)

In [ ]:
weight_and_posneg = weight_and_posneg.T

In [ ]:
weight_and_posneg.corr(method='pearson')

In [ ]:
weight_and_posneg.corr(method='pearson').to_csv('correlation5simple.csv', index=False)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(weight[0])), weight[0].cpu().numpy(), width=1, edgecolor=(0, 0, 0), label = "Feature weights by the classifier")
plt.bar(range(len(weight[1])), weight[1].cpu().numpy(), width=1, edgecolor=(0, 0, 0), color = 'teal', label = "Feature weights by the classifier")


plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Average weights of features at the first layer of the Classifier")
plt.legend(fontsize=12)

plt.show()

## Get predictions

In [ ]:
"""
Initialize the dataloaders. Use True for training, False for getting predictions
"""

train_tensor = TensorDataset(train_features, train_labels) 
train_dataloader = DataLoader(train_tensor, shuffle=False, batch_size=64)

test_tensor = TensorDataset(test_features, test_labels) 
eval_dataloader = DataLoader(test_tensor, shuffle=False, batch_size=64)

In [ ]:
"""
Predict for items from a dataloader
"""

def dataloader_predict(dataloader):
    predictions = torch.Tensor().to(device)

    model.eval()
    for batch in dataloader:
        with torch.no_grad():

            inputs, targets = batch
            inputs = inputs.to(device)
            logits = model(inputs.double())
            targets = targets.to(device)
            pred  = torch.argmax(logits, dim=-1)
            predictions = torch.cat((predictions, pred), 0)
    return predictions

predictions = dataloader_predict(train_dataloader)

print(predictions[:10])
print(train_labels[:10])

## Evaluate predictions

In [ ]:
import keras

keras.metrics.binary_accuracy(test_labels, predictions.to('cpu'), threshold=0.5)

In [ ]:
m = keras.metrics.Precision()
m.update_state(val_labels, predictions)
m.result().numpy()

In [ ]:
m = keras.metrics.Recall()
m.update_state(val_labels, predictions)
m.result().numpy()

In [ ]:
m = keras.metrics.AUC(num_thresholds=10)
m.update_state(test_labels, predictions)
m.result().numpy()

In [ ]:
from keras import metrics, backend

def f1_score(y_true, y_pred):
    m = metrics.Recall()
    m.update_state(y_true, y_pred)
    recall = m.result().numpy()
    m = metrics.Precision()
    m.update_state(y_true, y_pred)
    precision = m.result().numpy()
    return 2*((precision*recall)/(precision+recall+backend.epsilon()))
    
f1_score(test_labels, predictions)

## Pipeline for explaning predictions

In [ ]:
"""
This pipeline is used for LIME and thus needs numpy data for training and as input
"""

train_features = train_features.cpu().numpy() 
test_features = test_features.cpu().numpy()
train_labels = train_labels.cpu().numpy() 
test_labels = test_labels.cpu().numpy()
#val_features = val_features.cpu().numpy()
#val_labels = val_labels.cpu().numpy() 

In [ ]:
def pytpredict(input, model=model):
    """
    Predict for a single input, accepts a model WITHOUT pooling with input being a 2d numpy array
    """
    
    model = model.double()
    model.eval()
    predictions = []
    with torch.no_grad():
        input = torch.from_numpy(input)
        input = input.to(device)
        logits = model(input.double())
        for prediction in logits:
            prediction = F.softmax(logits).cpu().numpy()
            prediction = np.around(prediction, decimals=3)
            predictions.append(prediction)
        return(prediction)


In [ ]:
z = pytpredict(test_features[0])
print(z, z[0] + z[1])

## Get explanations for predictions

In [ ]:
print(test_labels[:10])

In [ ]:
print(train_labels[:10])

In [ ]:
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

In [ ]:
train_features = torch.mean(train_features, axis = 1)
test_features = torch.mean(test_features, axis = 1)
#val_features = torch.mean(val_features, axis = 1)

In [ ]:
"""
If usual conversion to Numpy doesn't work, try this
"""

test_labels = numpy.array(test_labels)
test_features = test_features.cpu().numpy()
val_labels = numpy.array(val_labels)
val_features = val_features.cpu().numpy()

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=train_features, mode='classification', training_labels=train_labels)

In [ ]:
explanation = explainer.explain_instance(train_features[0], pytpredict)


In [ ]:
explanation.show_in_notebook(show_table=True, show_all=False)

## Generate a series of explanations (Torch version)

In [ ]:
import pandas as pd

examples = pd.read_csv('100 examples to explain.csv')

In [ ]:
import random

n=100
sample_indices = random.sample(range(0, len(test_features)-1), n)
print(sample_indices)

In [ ]:
import random

n=100
sample_indices_train = random.sample(range(0, len(train_features)-1), n)
print(sample_indices_train)

In [ ]:
sample_indices = [4433, 7582, 2457, 2291, 6345, 6737, 2662, 3839, 5039, 7304, 6388, 2813, 2869, 6150, 2665, 1756, 4589, 
6286, 4001, 1596, 4817, 2261, 7292, 2595, 128, 3533, 2443, 6652, 3820, 796, 3518, 110, 575, 4142, 1950, 1216, 
2084, 2534, 3291, 4807, 2269, 3625, 1154, 5049, 5653, 5924, 2366, 3425, 1821, 3610, 348, 2181, 6432, 7560, 6981, 5382, 3898, 2889, 4019, 80, 
5578, 3515, 3151, 4652, 1838, 2447, 2319, 763, 2963, 1914, 5210, 5609, 2609, 3915, 4370, 6654, 5096, 3640, 3634, 3073, 3092, 
794, 695, 4275, 757, 4527, 5194, 421, 2121, 649, 2306, 3796, 4805, 6340, 2930, 3963, 7422, 4290, 736, 900]
sample_indices_train = [84, 19712, 32822, 32984, 40753, 14695, 30445, 16250, 2821, 6273, 25174, 2161, 27073, 11090, 43025, 39840, 
31322, 16112, 29403, 6409, 4967, 28014, 26647, 43307, 17783, 17906, 27331, 16215, 43685, 12797, 22501, 13549, 32395, 20889, 37749, 
37389, 25457, 24402, 29397, 34136, 28158, 30120, 24216, 17195, 20179, 44263, 21956, 4533, 7449, 26523, 1847, 41873, 18049, 1441, 
5721, 8534, 44336, 24693, 17965, 28259, 5536, 32428, 7676, 11491, 13094, 31243, 36254, 13872, 13704, 29117, 33016, 44498, 12866, 
19055, 11028, 42682, 22531, 28441, 13140, 17308, 39325, 20056, 41138, 8451, 9399, 9314, 23269, 27604, 12184, 35150, 26391, 28982, 
24946, 29393, 28172, 43355, 9124, 26595, 29999, 40449]

In [ ]:
occured_features_pos_label = []
occured_features_neg_label = []
occured_features_pos_pred = []
occured_features_neg_pred = []
occured_features_err = []
top_features = explanation.as_list()
stoplist = [">", "<", ".", "="]
print(top_features)
i = 4433
for f in top_features:
    print(111, f[0])
    candidates = f[0].split()
    print(000, candidates)
    for c in candidates:
        
        if not any(x in c for x in stoplist):
            feature = int(c)
            print('feaure', feature)
    if test_labels[i] == 1:
        occured_features_pos_label.append([feature, f])
    if test_labels[i] == 0:
        occured_features_neg_label.append([feature, f])
    if predictions[i] == 1:
        occured_features_pos_pred.append([feature, f])
    if predictions[i] == 0:
        occured_features_neg_pred.append([feature, f])
    if test_labels[i] != predictions[i]:
        occured_features_err.append([feature, f])

In [ ]:
stoplist = [">", "<", ".", "="]

occured_features_pos_pred = []
occured_features_neg_pred = []
occured_features_err = []
occured_features_pos_pred_dict = []
occured_features_neg_pred_dict = []

for y, i in enumerate(sample_indices_train):
    print(y)
    explanation = explainer.explain_instance(train_features[i], pytpredict)
    name = "C:\\Users\\ivank\\Documents\\BERT_projects\\Explanations_train\\exp_{}.html".format(i)
    explanation.save_to_file(name)
    top_features = explanation.as_list()
    print(top_features)
    for f in top_features:
        candidates = f[0].split()
        for c in candidates:
            if not any(x in c for x in stoplist):
                feature = int(c)
                print('feaure', feature)
        if predictions[i] == 1:
            occured_features_pos_pred.append(feature)
            occured_features_pos_pred_dict.append([feature, f])
        if predictions[i] == 0:
            occured_features_neg_pred.append(feature)
            occured_features_neg_pred_dict.append([feature, f])
        if train_labels[i] != predictions[i]:
            occured_features_err.append(feature)


In [ ]:
from collections import Counter
print("occured_features_pos_pred: ", Counter(occured_features_pos_pred))
print("occured_features_neg_pred: ", Counter(occured_features_neg_pred))
print("occured_features_err: ", Counter(occured_features_err))

#combined = occured_features_pos_label + occured_features_neg_label
#print("\n\nTotal top-features:", len(Counter(combined)), "/", len(combined))
#print(Counter(combined))


In [ ]:
#test
occured_features_pos_pred = {553: 51, 522: 35, 95: 33, 455: 31, 297: 27, 715: 21, 416: 18, 216: 17, 766: 14, 389: 13, 556: 13, 277: 12, 276: 10, 152: 9, 598: 9, 382: 9, 498: 8, 
137: 8, 585: 7, 289: 7, 36: 6, 524: 6, 195: 6, 85: 5, 461: 5, 746: 5, 100: 5, 559: 5, 502: 4, 384: 4, 443: 4, 205: 4, 125: 3, 138: 3, 31: 3, 438: 3, 673: 3, 653: 3, 582: 2, 607: 2,
 724: 2, 745: 2, 127: 2, 44: 2, 754: 2, 375: 2, 554: 2, 345: 2, 532: 2, 533: 2, 381: 2, 635: 2, 579: 2, 312: 2, 292: 2, 218: 2, 437: 2, 275: 2, 751: 2, 96: 2, 566: 2, 729: 2, 620: 2, 
 227: 2, 322: 1, 563: 1, 494: 1, 158: 1, 426: 1, 298: 1, 290: 1, 704: 1, 758: 1, 288: 1, 574: 1, 485: 1, 517: 1, 244: 1, 66: 1, 87: 1, 29: 1, 13: 1, 2: 1, 698: 1, 611: 1, 108: 1,
  113: 1, 243: 1, 173: 1, 399: 1, 454: 1, 613: 1, 240: 1, 733: 1, 341: 1, 86: 1, 222: 1, 626: 1, 471: 1, 519: 1, 77: 1, 428: 1, 667: 1, 463: 1, 712: 1, 258: 1, 175: 1, 293: 1, 752: 1, 
59: 1, 54: 1, 615: 1, 565: 1, 738: 1, 135: 1, 475: 1, 376: 1, 342: 1, 534: 1, 725: 1, 181: 1, 98: 1, 50: 1, 130: 1, 164: 1, 153: 1, 5: 1, 308: 1, 360: 1, 144: 1, 422: 1, 284: 1, 198: 1}

occured_features_neg_pred = {553: 31, 455: 29, 95: 26, 522: 23, 389: 22, 297: 20, 715: 20, 416: 14, 216: 12, 382: 11, 277: 10, 556: 9, 559: 8, 766: 7, 137: 7, 312: 6, 152: 6, 
276: 6, 85: 5, 36: 5, 275: 4, 375: 4, 289: 4, 402: 4, 215: 3, 673: 3, 751: 3, 471: 3, 329: 3, 585: 3, 195: 3, 620: 3, 524: 3, 443: 3, 598: 3, 381: 2, 127: 2, 653: 2, 540: 2, 502: 2,
 66: 2, 175: 2, 218: 2, 53: 2, 164: 2, 474: 2, 41: 2, 399: 2, 712: 2, 498: 2, 100: 2, 441: 2, 745: 2, 59: 2, 138: 2, 516: 2, 205: 2, 461: 2, 563: 1, 13: 1, 463: 1, 475: 1, 239: 1, 
 344: 1, 353: 1, 188: 1, 144: 1, 31: 1, 110: 1, 557: 1, 737: 1, 532: 1, 0: 1, 408: 1, 32: 1, 725: 1, 512: 1, 286: 1, 658: 1, 161: 1, 393: 1, 676: 1, 404: 1, 572: 1, 348: 1, 724: 1, 
 582: 1, 384: 1, 321: 1, 499: 1, 194: 1, 309: 1, 20: 1, 372: 1, 328: 1, 367: 1, 153: 1, 255: 1, 636: 1, 200: 1, 73: 1, 185: 1, 523: 1, 517: 1, 298: 1, 351: 1, 258: 1, 413: 1, 135: 1, 
 593: 1, 478: 1, 345: 1, 212: 1, 130: 1, 60: 1, 149: 1, 278: 1, 44: 1, 354: 1, 292: 1, 533: 1, 223: 1, 141: 1, 597: 1, 534: 1, 424: 1, 547: 1, 727: 1, 469: 1, 181: 1, 226: 1, 703: 1, 357: 1, 635: 1, 158: 1, 392: 1}


#train
"""occured_features_pos_pred = {135: 44, 498: 32, 47: 28, 553: 22, 216: 21, 297: 17, 585: 17, 153: 16, 137: 16, 766: 14, 294: 13, 275: 12, 638: 12, 727: 11, 715: 11, 758: 10, 416: 10, 517: 8, 502: 8, 378: 8, 
653: 7, 345: 7, 130: 6, 596: 6, 222: 6, 532: 6, 412: 6, 646: 6, 608: 6, 114: 6, 127: 5, 635: 5, 382: 5, 241: 4, 215: 4, 66: 4, 295: 4, 243: 4, 401: 4, 729: 4, 566: 3, 286: 3, 438: 3, 624: 3, 265: 3, 613: 3, 
253: 3, 708: 3, 85: 3, 478: 3, 492: 3, 522: 2, 573: 2, 176: 2, 540: 2, 121: 2, 304: 2, 421: 2, 159: 2, 337: 2, 484: 2, 178: 2, 563: 2, 361: 2, 181: 2, 334: 2, 661: 2, 523: 1, 289: 1, 1: 1, 639: 1, 
544: 1, 480: 1, 166: 1, 357: 1, 156: 1, 16: 1, 651: 1, 163: 1, 761: 1, 543: 1, 597: 1, 11: 1, 524: 1, 384: 1, 161: 1, 360: 1, 175: 1, 410: 1, 724: 1, 172: 1, 106: 1, 226: 1, 701: 1, 329: 1, 691: 1, 
168: 1, 647: 1, 154: 1, 35: 1, 736: 1, 533: 1, 618: 1, 255: 1, 399: 1, 270: 1, 636: 1, 370: 1, 276: 1, 108: 1, 31: 1, 94: 1, 753: 1, 408: 1, 299: 1, 198: 1, 245: 1}

occured_features_neg_pred = {135: 25, 553: 22, 498: 20, 294: 16, 297: 14, 47: 13, 608: 13, 137: 12, 766: 12, 275: 11, 758: 10, 416: 10, 585: 10, 153: 9, 216: 8, 638: 8, 540: 7, 215: 7, 378: 7, 
176: 7, 596: 6, 517: 6, 624: 6, 653: 6, 85: 6, 127: 6, 295: 6, 94: 6, 712: 5, 243: 5, 502: 5, 727: 5, 532: 5, 635: 4, 130: 4, 412: 4, 566: 4, 159: 4, 729: 4, 444: 4, 646: 4, 401: 3, 114: 3, 478: 3, 
426: 3, 438: 3, 241: 3, 563: 3, 286: 3, 222: 3, 636: 2, 715: 2, 745: 2, 337: 2, 484: 2, 265: 2, 181: 2, 724: 2, 108: 2, 528: 2, 625: 2, 676: 2, 691: 2, 31: 2, 345: 2, 615: 1, 4: 1, 370: 1, 708: 1, 
524: 1, 658: 1, 329: 1, 522: 1, 685: 1, 687: 1, 173: 1, 255: 1, 273: 1, 387: 1, 321: 1, 382: 1, 379: 1, 665: 1, 324: 1, 251: 1, 562: 1, 572: 1, 253: 1, 357: 1, 629: 1, 613: 1, 603: 1, 259: 1, 573: 1, 
556: 1, 597: 1, 258: 1, 66: 1, 10: 1, 761: 1, 58: 1, 225: 1, 178: 1, 375: 1, 460: 1, 583: 1, 399: 1, 334: 1, 516: 1, 72: 1, 592: 1, 747: 1}"""

In [ ]:
# Initializing from dictionaries

from collections import Counter
combined = Counter(occured_features_pos_pred) + Counter(occured_features_neg_pred)
#pos_counter, neg_counter = occured_features_pos_pred, occured_features_neg_pred

In [ ]:
# if initializing directly after obtaining LIME statistics

combined_counter = dict(Counter(combined))
pos_counter = dict(Counter(occured_features_pos_pred))
neg_counter = dict(Counter(occured_features_neg_pred))
#err_counter = dict(Counter(occured_features_err))


In [ ]:
combined_counter = dict(sorted(combined_counter.items(), key=lambda item: item[1], reverse=True))
pos_counter = dict(sorted(pos_counter.items(), key=lambda item: item[1], reverse=True))
neg_counter = dict(sorted(neg_counter.items(), key=lambda item: item[1], reverse=True))
#err_counter = dict(sorted(err_counter.items(), key=lambda item: item[1], reverse=True))

In [ ]:
#combined_counter.keys()
top_combined_counter = [f for f in combined_counter.keys() if combined_counter[f] > 10]
top_pos_counter = [f for f in pos_counter.keys() if pos_counter[f] > 10]
top_neg_counter = [f for f in neg_counter.keys() if neg_counter[f] > 10]
#top_err_counter = [f for f in err_counter.keys() if err_counter[f] > 1]

In [ ]:
pos_neg = set(top_pos_counter).intersection(set(top_neg_counter)) #which features among most popular are common among positive and negative
pos_neg

In [ ]:
#for Torch model data

pos_neg = set(top_pos_counter).intersection(set(top_neg_counter)) #which features among most popular are common among positive and negative
pos_neg

In [ ]:
pos_err = set(top_pos_counter).intersection(set(top_err_counter))
neg_err = set(top_neg_counter).intersection(set(top_err_counter))
print(pos_err, neg_err)
print(pos_err.intersection(neg_err))

In [ ]:
comb_err = set(top_combined_counter).intersection(set(top_err_counter))
print(comb_err)

In [ ]:
pos_counter[378]

## Plot the explanations statistics

In [ ]:
from matplotlib import pyplot as plt

combined_counter_masked1 = combined_counter.copy()
for key, value in combined_counter_masked1.items():
    if key in pos_counter.keys():
        combined_counter_masked1[key] = pos_counter[key]
    else:
        combined_counter_masked1[key] = 0

combined_counter_masked2 = combined_counter.copy()
for key, value in combined_counter_masked2.items():
    if key in set(top_pos_counter):
        combined_counter_masked2[key] = pos_counter[key]
    else:
        combined_counter_masked2[key] = 0

plt.figure(figsize=(20, 10))
plt.bar(range(len(combined_counter.keys())), combined_counter.values(), width=1, edgecolor=(0, 0, 0), label = "Total top-feature counts")
plt.bar(range(len(combined_counter_masked1.keys())), combined_counter_masked1.values(), width=1, edgecolor=(0, 0, 0), color = 'peachpuff', label = "Top-feature counts for predicted label \"1\"")
plt.bar(range(len(combined_counter_masked2.keys())), combined_counter_masked2.values(), width=1, edgecolor=(0, 0, 0), color = 'coral', label = "Top-features with > 10 counts for predicted label \"1\"")



plt.xlabel("Features")
plt.ylabel("Counts in explanations")
plt.title("Counts of top-10 features in 100 explanations + top features for label \"1\"")
plt.xticks(range(20), list(combined_counter.keys())[:20], rotation=90, size=7)
plt.legend(fontsize=12)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
"""
combined_counter_masked = combined_counter.copy()
for key, value in combined_counter_masked.items():
    if key in set(top_neg_counter):
        combined_counter_masked[key] = neg_counter[key]
    else:
        combined_counter_masked[key] = 0
"""

combined_counter_masked1 = combined_counter.copy()
for key, value in combined_counter_masked1.items():
    if key in neg_counter.keys():
        combined_counter_masked1[key] = neg_counter[key]
    else:
        combined_counter_masked1[key] = 0

combined_counter_masked2 = combined_counter.copy()
for key, value in combined_counter_masked2.items():
    if key in set(top_neg_counter):
        combined_counter_masked2[key] = neg_counter[key]
    else:
        combined_counter_masked2[key] = 0

plt.figure(figsize=(20, 10))
plt.bar(range(len(combined_counter.keys())), combined_counter.values(), width=1, edgecolor=(0, 0, 0))
plt.bar(range(len(combined_counter_masked1.keys())), combined_counter_masked1.values(), width=1, edgecolor=(0, 0, 0), color = 'palegreen', label = "Top-feature counts for predicted label \"0\"")
plt.bar(range(len(combined_counter_masked2.keys())), combined_counter_masked2.values(), width=1, edgecolor=(0, 0, 0), color = 'limegreen', label = "Top-features with > 10 counts for predicted label \"0\"")
plt.xlabel("Features")
plt.ylabel("Counts in explanations")
plt.title("Counts of top-10 features in 100 explanations + top features for label \"0\"")
plt.xticks(range(20), list(combined_counter.keys())[:20], rotation=90, size=7)
plt.legend(fontsize=12)
plt.show()

In [ ]:
from matplotlib import pyplot as plt

combined_counter_masked = combined_counter.copy()
for key, value in combined_counter_masked.items():
    if key in comb_err:
        combined_counter_masked[key] = err_counter[key] 
    else:
        combined_counter_masked[key] = 0

plt.figure(figsize=(20, 10))
plt.bar(range(len(combined_counter.keys())), combined_counter.values(), width=1, edgecolor=(0, 0, 0))
plt.bar(range(len(combined_counter_masked.keys())), combined_counter_masked.values(), width=1, edgecolor=(0, 0, 0), color = 'red', label = "10 most common top-features for erroneous classification")

plt.xlabel("Features")
plt.ylabel("Counts in explanations")
plt.title("Counts of top-10 features in 100 explanations + top features for mistakes")
plt.xticks(range(20), list(combined_counter.keys())[:20], rotation=90, size=7)
plt.legend(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(combined_counter.keys(), combined_counter.values(), width=2)
plt.xlabel("Features ordered by number")
plt.ylabel("Counts in explanations")
plt.title("Counts of top-10 features in 100 explanations in raw order")
plt.show()